In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
predictor = TabularPredictor(label='class').fit(train_data=train_data)
predictions = predictor.predict(test_data)

In [ ]:
predictions

0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
df_train = TabularDataset('data/train.csv')
df_test = TabularDataset('data/test.csv')

Loaded data from: data/train.csv | Columns = 31 / 31 | Rows = 200000 -> 200000
Loaded data from: data/test.csv | Columns = 30 / 30 | Rows = 50000 -> 50000


In [ ]:
# 对缺失值进行填充
df_train.fillna(-1, inplace=True)
df_test.fillna(-1, inplace=True)

df_train.isnull().sum()

ID        0
区域1       0
区域2       0
区域3       0
街道        0
上传日期      0
房屋租金      0
服务费       0
供暖费用      0
电力基础价格    0
有阳台       0
没有停车位     0
有厨房       0
有地窖       0
居住面积      0
房屋状况      0
内饰质量      0
可带宠物      0
加热类型      0
有电梯       0
房屋类型      0
邮政编码      0
房间数量      0
所处楼层      0
建筑楼层      0
有花园       0
最后翻新年份    0
是新建筑      0
建成年份      0
价格趋势      0
上传图片数     0
dtype: int64

In [ ]:
import re
# 对特征的类型进行处理
# 将上传日期变成年份和月份
upload_date = df_train['上传日期'].values
year_data = []
month_data = []
month_to_num = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 
                'May':5, 'Jun':6, 'Jul':7, 'Aug':8,
                'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
for i in upload_date:
    year_data.append(int(re.findall(r"[0-9]*[.]?[0-9]+", i)[0]))
    month_data.append(month_to_num[re.findall(r"[a-zA-Z]+\’*[a-z]*", i)[0]])

df_train['上传日期_年'] = year_data
df_train['上传日期_月'] = month_data
df_train.drop(columns=['上传日期'], inplace=True)

# 将特征值为布尔值变成数字
features_bool = ['有阳台', '有厨房', '有地窖', '有电梯', '有花园', '是新建筑']
df_train[features_bool] = df_train[features_bool].astype(int)

pet_dict = {'no':0, "yes":1, "negotiable":2}
pet_num = []
pet_bool = df_train['可带宠物'].values
for pet in pet_bool:
    if pet == -1:
        pet_num.append(-1)
    else:
        pet_num.append(pet_dict[pet])

df_train['可带宠物'] = pet_num

# 对测试集特征的类型进行处理
# 将上传日期变成年份和月份
upload_date = df_test['上传日期'].values
year_data = []
month_data = []
month_to_num = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 
                'May':5, 'Jun':6, 'Jul':7, 'Aug':8,
                'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
for i in upload_date:
    year_data.append(int(re.findall(r"[0-9]*[.]?[0-9]+", i)[0]))
    month_data.append(month_to_num[re.findall(r"[a-zA-Z]+\’*[a-z]*", i)[0]])

df_test['上传日期_年'] = year_data
df_test['上传日期_月'] = month_data
df_test.drop(columns=['上传日期'], inplace=True)

# 将特征值为布尔值变成数字
features_bool = ['有阳台', '有厨房', '有地窖', '有电梯', '有花园', '是新建筑']
df_test[features_bool] = df_test[features_bool].astype(int)

pet_dict = {'no':0, "yes":1, "negotiable":2}
pet_num = []
pet_bool = df_test['可带宠物'].values
for pet in pet_bool:
    if pet == -1:
        pet_num.append(-1)
    else:
        pet_num.append(pet_dict[pet])

df_test['可带宠物'] = pet_num

In [ ]:
# 去除异常值
index_zero = df_train.loc[df_train['房屋租金'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero)))
df_train.drop(index = index_zero, inplace=True)
df_train = df_train.reset_index(drop=True)

index_zero_sq = df_train.loc[df_train['居住面积'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero_sq)))
df_train.drop(index = index_zero_sq, inplace=True)
df_train = df_train.reset_index(drop=True)

# 房屋租金这里的异常值，个人认为4000会比较好，但是由于一开始设置成了3000，所以目前以3000为基准进行测试
index_zero = df_train.loc[df_train['房屋租金'] > 3000].index
print("房租价格等于0的数量有{0}".format(len(index_zero)))
df_train.drop(index = index_zero, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[df_train['居住面积'] > 300].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[df_train['服务费'] >750].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格等于0的数量有68
房租价格等于0的数量有52
房租价格等于0的数量有997
房租价格超过50的数量有：45
房租价格超过50的数量有：64


In [ ]:
predictor = TabularPredictor(label='房屋租金').fit(train_data=df_train,presets='best_quality')
predictions = predictor.predict(df_test)
df_test['房屋租金'] = predictions
df_test[['ID', '房屋租金']].to_csv('submit/autoML2.csv', index=False, encoding='utf-8')

No path specified. Models will be saved in: "AutogluonModels/ag-20221011_041344\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221011_041344\"
AutoGluon Version:  0.5.2
Python Version:     3.8.13
Operating System:   Windows
Train Data Rows:    198774
Train Data Columns: 31
Label Column: 房屋租金
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3000.0, 1.0, 628.98157, 422.64632)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generato

[1000]	valid_set's rmse: 144.956
[2000]	valid_set's rmse: 137.465
[3000]	valid_set's rmse: 133.733
[4000]	valid_set's rmse: 131.196
[5000]	valid_set's rmse: 129.493
[6000]	valid_set's rmse: 127.89
[7000]	valid_set's rmse: 127.191
[8000]	valid_set's rmse: 126.363
[9000]	valid_set's rmse: 125.466
[10000]	valid_set's rmse: 124.927


	-124.9197	 = Validation score   (-root_mean_squared_error)
	28.53s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 129.395
[2000]	valid_set's rmse: 124.167
[3000]	valid_set's rmse: 121.334
[4000]	valid_set's rmse: 119.339
[5000]	valid_set's rmse: 118.619
[6000]	valid_set's rmse: 118.149
[7000]	valid_set's rmse: 117.491
[8000]	valid_set's rmse: 116.759
[9000]	valid_set's rmse: 116.279
[10000]	valid_set's rmse: 115.997


	-115.9639	 = Validation score   (-root_mean_squared_error)
	27.74s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-158.4858	 = Validation score   (-root_mean_squared_error)
	46.26s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	-117.9617	 = Validation score   (-root_mean_squared_error)
	131.07s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-165.7041	 = Validation score   (-root_mean_squared_error)
	24.99s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-155.9112	 = Validation score   (-root_mean_squared_error)
	144.41s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-117.9006	 = Validation score   (-root_mean_squared_error)
	53.73s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-158.1097	 = Validation score   (-root_mean_squared_error)
	381.16s	 = Training   runtime
	0

[1000]	valid_set's rmse: 124.203
[2000]	valid_set's rmse: 119.903
[3000]	valid_set's rmse: 118.394
[4000]	valid_set's rmse: 117.381
[5000]	valid_set's rmse: 116.761
[6000]	valid_set's rmse: 116.193
[7000]	valid_set's rmse: 115.956
[8000]	valid_set's rmse: 115.708
[9000]	valid_set's rmse: 115.558
[10000]	valid_set's rmse: 115.372


	-115.3722	 = Validation score   (-root_mean_squared_error)
	46.54s	 = Training   runtime
	0.24s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-113.0131	 = Validation score   (-root_mean_squared_error)
	0.22s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 911.02s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20221011_041344\")


In [ ]:
# df_test = pd.read_csv('data/test.csv')
# df_test

,ID,区域1,区域2,区域3,街道,上传日期,服务费,供暖费用,电力基础价格,有阳台,...,邮政编码,房间数量,所处楼层,建筑楼层,有花园,最后翻新年份,是新建筑,建成年份,价格趋势,上传图片数
0,200000,5,9,2360,13380,Feb20,250.00,NaN,NaN,True,...,2435,3.0,NaN,NaN,False,2019.0,False,NaN,3.00,5
1,200001,1,175,8202,25,Sep18,130.00,NaN,90.76,True,...,7192,1.0,NaN,NaN,True,NaN,False,2013.0,5.06,6
2,200002,9,307,1883,1,Feb20,200.00,NaN,NaN,True,...,1982,2.0,-1.0,NaN,False,NaN,False,2015.0,3.14,5
3,200003,9,87,123,5819,May19,250.00,NaN,NaN,True,...,125,4.0,0.0,3.0,False,2019.0,False,1969.0,2.57,10
4,200004,13,42,2316,3376,May19,160.00,NaN,NaN,True,...,2390,2.5,NaN,NaN,False,NaN,False,1965.0,2.52,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,249995,12,169,423,50318,Feb20,28.00,38.00,NaN,True,...,439,1.0,1.0,5.0,True,2013.0,False,1983.0,0.19,9
49996,249996,0,0,1845,1363,Oct19,150.00,NaN,NaN,False,...,1945,3.0,NaN,NaN,False,NaN,False,1967.0,3.85,10
49997,249997,6,146,1639,32166,Oct19,245.27,106.45,NaN,True,...,709,4.0,NaN,NaN,False,NaN,True,2019.0,3.68,11
49998,249998,5,9,224,26407,Feb20,272.00,NaN,NaN,True,...,228,4.0,2.0,4.0,False,NaN,False,2015.0,4.24,22


In [ ]:
# 导出csv文件
df_test['房屋租金'] = predictions
df_test[['ID', '房屋租金']].to_csv('submit/autoML2.csv', index=False, encoding='utf-8')

In [ ]:
features = df_train.columns.drop(['房屋租金'])
features

Index(['ID', '区域1', '区域2', '区域3', '街道', '服务费', '供暖费用', '电力基础价格', '有阳台',
       '没有停车位', '有厨房', '有地窖', '居住面积', '房屋状况', '内饰质量', '可带宠物', '加热类型', '有电梯',
       '房屋类型', '邮政编码', '房间数量', '所处楼层', '建筑楼层', '有花园', '最后翻新年份', '是新建筑', '建成年份',
       '价格趋势', '上传图片数', '上传日期_年', '上传日期_月'],
      dtype='object')

In [ ]:
X_train = df_train[features]
X_train

,ID,区域1,区域2,区域3,街道,服务费,供暖费用,电力基础价格,有阳台,没有停车位,...,所处楼层,建筑楼层,有花园,最后翻新年份,是新建筑,建成年份,价格趋势,上传图片数,上传日期_年,上传日期_月
0,0,0,0,0,0,140.0,-1.0,-1.00,1,1.0,...,2.0,3.0,0,-1.0,0,1971.0,4.62,6,19,10
1,1,1,1,1,1,120.0,-1.0,-1.00,0,-1.0,...,-1.0,-1.0,0,-1.0,0,1967.0,5.95,7,19,10
2,2,0,2,2,2,-1.0,-1.0,-1.00,0,-1.0,...,1.0,1.0,0,-1.0,0,2018.0,3.83,10,20,2
3,3,2,3,3,3,153.0,-1.0,-1.00,1,1.0,...,2.0,3.0,0,-1.0,0,1972.0,5.24,7,19,5
4,4,3,4,4,1,350.0,-1.0,-1.00,1,-1.0,...,2.0,2.0,1,-1.0,0,-1.0,5.63,14,19,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198769,199995,6,71,4704,536,120.0,-1.0,-1.00,0,1.0,...,-1.0,-1.0,1,-1.0,0,1710.0,2.05,1,19,5
198770,199996,8,11,157,2531,131.0,-1.0,-1.00,1,-1.0,...,2.0,6.0,0,-1.0,0,1969.0,8.36,7,20,2
198771,199997,5,9,596,11079,82.0,43.0,90.76,1,-1.0,...,0.0,-1.0,0,-1.0,0,1980.0,5.66,10,18,9
198772,199998,1,262,5697,431,228.0,-1.0,-1.00,1,1.0,...,2.0,3.0,0,2017.0,0,-1.0,2.92,11,19,10


In [ ]:
y_train = df_train['房屋租金']
y_train

0          569.0
1          560.0
2          720.0
3          695.0
4         1200.0
           ...  
198769     220.0
198770     534.0
198771     225.0
198772    1142.0
198773     480.0
Name: 房屋租金, Length: 198774, dtype: float64